# 04-Spam-Classifier

It's time to make our first real Machine Learning application of NLP: a spam classifier!

A spam classifier is a Machine Learning model that classifier texts (email or SMS) into two categories: Spam (1) or legitimate (0).

To do that, we will reuse our knowledge: we will apply preprocessing and BOW (Bag Of Words) on a dataset of texts.
Then we will use a classifier to predict to which class belong a new email/SMS, based on the BOW.

First things first: import the needed libraries.

In [2]:
# Import NLTK and all the needed libraries
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

Load now the dataset in *spam.csv* using pandas. Use the 'latin-1' encoding as loading option.

In [3]:
# TODO: Load the dataset 

data = pd.read_csv('spam.csv')
data.head()

,Class,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


As usual, I suggest you to explore a bit this dataset.

In [4]:
# TODO: explore the dataset

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Class    5572 non-null   object
 1   Message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


So as you see we have a column containing the labels, and a column containing the text to classify.

We will begin by doing the usual preprocessing: tokenization, punctuation removal and lemmatization.

In [7]:
# TODO: Perform preprocessing over all the text

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_txt(text):
    return[lemmatizer.lemmatize(w) for w in text]


data['Message'] = data['Message'].str.replace('[^a-zA-Z0-9]','')
data['tokens'] = data['Message'].apply(word_tokenize)
data['tokens'] = data['tokens'].apply(lemmatize_txt)

print(data['tokens'])

C:\Users\dudes\AppData\Local\Temp\ipykernel_21344\3031125482.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Message'] = data['Message'].str.replace('[^a-zA-Z0-9]','')


0       [GountiljurongpointcrazyAvailableonlyinbugisng...
1                                    [OklarJokingwifuoni]
2       [Freeentryin2awklycomptowinFACupfinaltkts21stM...
3                     [UdunsaysoearlyhorUcalreadythensay]
4       [NahIdontthinkhegoestousfhelivesaroundherethough]
                              ...                        
5567    [Thisisthe2ndtimewehavetried2contactuUhavewont...
5568                        [Willbgoingtoesplanadefrhome]
5569          [PitywasinmoodforthatSoanyothersuggestions]
5570    [TheguydidsomebitchingbutIactedlikeidbeinteres...
5571                               [RoflItstruetoitsname]
Name: tokens, Length: 5572, dtype: object


Ok now we have our preprocessed data. Next step is to do a BOW.

In [8]:
# TODO: compute the BOW

from sklearn.feature_extraction.text import CountVectorizer

vectorize = CountVectorizer(stop_words = 'english')

bow = vectorize.fit_transform(data['Message']).toarray()

Then make a new dataframe as usual to have a visual idea of the words used and their frequencies.

In [14]:
# TODO: Make a new dataframe with the BOW

df_bow = pd.DataFrame(data=bow, columns = vectorize.get_feature_names_out())

df_bow.head()

,07732584351rodgerburnsmsgwetriedtocallyoureyourreplytooursmsforafreenokiamobilefreecamcorderpleasecallnow08000930705fordeliverytomorrow,08714712388between10am7pmcost10p,09066362231urgentyourmobileno07xxxxxxxxxwona2000bonuscallerprizeon020603thisisthe2ndattempttoreachyoucall09066362231asap,0anetworksallowcompaniestobillforsmssotheyareresponsiblefortheirsuppliersjustasashophastogiveaguaranteeonwhattheysellbg,1000sflirtingnowtxtgirlorblokeurnameageeggirlzoe18to8007tojoinandgetchatting,1000sofgirlsmanylocal2uwhorvirgins2thisrready24filureverysexualneedcanu4filtheirstextcuteto69911150pm,100datingservicecall09064012103box334sk38ch,10minlaterk,120thatcallcostwhichiguessisntbadmissyaneedyawantyaloveya,123congratulationsinthisweekscompetitiondrawuhavewonthe1450prizetoclaimjustcall09050002311b4280703tcsstopsms08718727868over18only150ppm,...,yupok,yupokigohomelookatthetimingsthenimsgagainxuhuigoingtolearnon2ndmaytoobutherlessonisat8am,yupokthanx,yupsongbronocreativenevatestqualityhesaidcheckreviewonline,yupthkofuosoboringwat,yupwunbelievewatureallynevacemsgisentshuhui,yupziveoredibookedslots4myweekendsliao,ywantogotherecdoctor,ywhereuatdogbreathitsjustsoundinglikejancthatsal,zoeitjusthitme2imfuckingshitinmyselfildefotrymyhardest2cum2morowluvumillionslekdog
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's check what is the most used word in the spam category and the non spam category.

There are two steps: first add the class to the BOW dataframe. Second, filter on a class, sum all the values and print the most frequent one.

In [15]:
# TODO: print the most used word in the spam and non spam category

tmp = df_bow

tmp['Class'] = data['Class']

most_frequent_spam = tmp[tmp['Class'] == 'spam'].drop(columns='Class').sum().idxmax()
most_frequent_non_spam = tmp[tmp['Class'] == 'ham'].drop(columns='Class').sum().idxmax()

print('most frequent spam word :',most_frequent_spam )
print('most frequent non spam word :',most_frequent_non_spam )

most frequent spam word : camerayouareawardedasipixdigitalcameracall09061221066frommlandlinedeliverywithin28days
most frequent non spam word : sorryillcalllater


You should find that the most frequent spam word is 'free', not so surprising, right?

Now we can make a classifier based on our BOW. We will use a simple logistic regression here for the example.

You're an expert, you know what to do, right? Split the data, train your model, predict and see the performance.

In [23]:
# TODO: Perform a classification to predict whether a message is a spam or not

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import precision_score

le = preprocessing.LabelEncoder()
le.fit(data['Class'])

print(le.transform(data['Class']))

train_x, test_x, train_y, test_y = train_test_split(df_bow.drop(columns='Class'), le.transform(data['Class']), test_size=0.2, random_state=42)

classifier = LogisticRegression()

classifier.fit(train_x, train_y)

predict = classifier.predict(test_x)

acc_score = accuracy_score(predict, test_y)
prec_score = precision_score(predict, test_y)
print('Accuracy Score :', acc_score)
print('Precision Score :', prec_score)

[0 0 1 ... 0 0 0]
Accuracy Score : 0.8654708520179372
Precision Score : 0.0


What precision do you get? Check by hand on some samples where it did predict well to check what could go wrong

The Accuracy score of the model is 0.865
The precision score of the model is 0

Try to use other models and try to improve your results.